In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.font_manager import FontProperties

# Constants
R = 8.314  # Gas constant in kcal/(mol*K)
T = 300  # Temperature in Kelvin

# Function to create a custom colormap
def create_custom_cmap(cmap_name, colors):
    return LinearSegmentedColormap.from_list(cmap_name, colors, N=256)

# Function to preprocess data to ensure it is within the specified range
def preprocess_data(data, range_min=-180, range_max=180):
    """Ensure data is within the specified range, wrapping around for angle data."""
    return ((data - range_min) % (range_max - range_min) + range_min)

# Function to read data from a single CSV file
def read_data(file, usecols1, usecols2, skip_header=1):
    data1 = np.genfromtxt(file, delimiter=',', usecols=usecols1, skip_header=skip_header)
    data2 = np.genfromtxt(file, delimiter=',', usecols=usecols2, skip_header=skip_header)
    
    # Preprocess data to ensure it is within the -180 to 180 range
    data1 = preprocess_data(data1, -180, 180)
    data2 = preprocess_data(data2, -180, 180)
    
    return data1, data2

# Function to bin the data
def bin_data(data1, data2, num_bins1, num_bins2, range1, range2):
    hist, xedges, yedges = np.histogram2d(data1, data2, bins=[num_bins1, num_bins2], range=[range1, range2])
    probabilities = hist / np.sum(hist)
    return probabilities, xedges, yedges

# Function to compute energy difference between bins
def compute_energy(probabilities):
    energy = np.zeros(probabilities.shape)
    for i in range(probabilities.shape[0]):
        for j in range(probabilities.shape[1]):
            if probabilities[i, j] > 0:
                energy[i, j] = -R/1000 * T * 0.238846 * np.log(probabilities[i, j])
            else:
                energy[i, j] = np.inf
    return energy

# Function to plot the combined 3D energy landscape and 2D energy map
def plot_energy_combined(energy, xedges, yedges, filename, custom_cmap):
    font_prop = FontProperties(family='Times New Roman', weight='bold', size=14)
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    x = np.linspace(xedges[0], xedges[-1], energy.shape[1])
    y = np.linspace(yedges[0], yedges[-1], energy.shape[0])
    X, Y = np.meshgrid(x, y)
    
    surf = ax.plot_surface(X, Y, energy.T, cmap=custom_cmap, edgecolor='none', alpha=0.8)
    
    ax.set_xlabel('χ1 (°)', fontproperties=font_prop)
    ax.set_ylabel('D3 (Å)', fontproperties=font_prop)
    ax.set_zlabel('Free Energy (kcal/mol)', fontproperties=font_prop)
    ax.set_title('HID378 Free Energy Landscape', fontsize=16, fontweight='bold')
    
    cbar = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10)
    cbar.ax.tick_params(labelsize=12)
    cbar.set_label('Free Energy (kcal/mol)', fontproperties=font_prop)
    
    cont = ax.contourf(X, Y, energy.T, zdir='z', offset=energy.min(), cmap=custom_cmap, alpha=0.5)
    
    # Set X-axis range explicitly
    ax.set_xlim([-180, 180])
    
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()

# Main function to execute the workflow
def main(file, usecols1, usecols2, num_bins1, num_bins2, range1, range2, filename, skip_header=1):
    data1, data2 = read_data(file, usecols1, usecols2, skip_header)
    
    # Print preprocessed data samples
    print("Preprocessed Data1 (χ1) sample:", data1[:10])
    print("Preprocessed Data2 (D3) sample:", data2[:10])
    
    probabilities, xedges, yedges = bin_data(data1, data2, num_bins1, num_bins2, range1, range2)
    energy = compute_energy(probabilities)
    custom_cmap = create_custom_cmap('blue_white_red', ['blue', 'white', 'red'])
    plot_energy_combined(energy, xedges, yedges, filename, custom_cmap)

if __name__ == "__main__":
    file = 'combined_all.csv'
    usecols1 = 2
    usecols2 = 1
    num_bins1 = 80
    num_bins2 = 80
    range1 = [-180, 180]  # Corrected range
    range2 = [2, 8]
    filename = 'HIS378_free_energy_landscape.jpg'
    main(file, usecols1, usecols2, num_bins1, num_bins2, range1, range2, filename)
